The quote from Ritzwoller & Romano 2021:
> ...multiple testing procedures reveal that one shooter exhibits a shooting pattern significantly inconsistent with randomness – supplying strong evidence that basketball shooting is not random for all shooters all of the time. However, we find that the evidence against randomness in this experiment is limited to this shooter

link: https://www.restud.com/paper/uncertainty-in-the-hot-hand-fallacy-detecting-streaky-alternatives-to-random-bernoulli-sequences/

# Skip to (3) for results

## 1. Three Stata rclass commands:  (skip over this for results)
1. Generating random data
2. Running a fixed effect regression coefficient (biased)
3. Running a biased-corrected test on the regression coefficient

In [7]:

*Helper code:
{

cap program drop genshots_GVT
program define genshots_GVT
	syntax [, fg(string) nshots(integer 100) nsids(integer 26)]
	*This code regenerates GVT's shot outcomes, randomly 
	clear
	local obs=`nshots'*`nsids'
	set obs `obs'
	gen rand1=runiform()
	gen period=mod(_n-1,`nshots')+1
	gen sid=100+floor((_n-period)/`nshots')+1
	replace sid=200+sid-100-14 if sid>114
	
	*These shooters had empty observations, because they ended early
	drop if period>90 & sid==104
	drop if period>75& sid==107
	drop if period>50 & sid==108
	
	gen fg=.
	if "`fg'"=="design"{
		replace fg=.5
	}
	else if "`fg'"=="actual"{
		replace fg=.54 if sid==101
		replace fg=.35 if sid==102
		replace fg=.6 if sid==103
		replace fg=.4 if sid==104
		replace fg=.42 if sid==105
		replace fg=.57 if sid==106
		replace fg=.56 if sid==107
		replace fg=.5 if sid==108
		replace fg=.54 if sid==109
		replace fg=.6 if sid==110
		replace fg=.58 if sid==111
		replace fg=.44 if sid==112
		replace fg=.61 if sid==113
		replace fg=.59 if sid==114
		replace fg=.48 if sid==201
		replace fg=.34 if sid==202
		replace fg=.39 if sid==203
		replace fg=.32 if sid==204
		replace fg=.36 if sid==205
		replace fg=.46 if sid==206
		replace fg=.41 if sid==207
		replace fg=.53 if sid==208
		replace fg=.45 if sid==209
		replace fg=.46 if sid==210
		replace fg=.53 if sid==211
		replace fg=.25 if sid==212
	}
	else{
		display "ERROR: control flow issues"
		error
		exit
	}
	
	gen make=(rand1<=fg)
end

cap program drop anGVT_fe
program define anGVT_fe, rclass
	syntax [, fg(string) k(integer 3) command(string) ]
	*This code regenerates GVT's shot outcomes, randomly 
	*Then runs a fixed effect regression (biased), and returns the coefficient estimate 
	*hitting k or more in row
	
	clear
	genshots_GVT, fg(`fg')
	
	*Flexible, to add any command here:
	`command'
	
	*Build streak condition statement
	local make="make[_n-1]==1"
	local miss="make[_n-1]==0"
	forvalues i=2(1)`k'{
		local make="`make' & make[_n-`i']==1"
		local miss="`miss' & make[_n-`i']==0"
	}
	
	*Generate the regressor:
	gen dMade`k'ormore=`make' if period>`k'
	gen dMissed`k'ormore=`miss' if period>`k'
	
	*fixed effect regression 
	reg make dMade`k'ormore i.sid if dMade`k'ormore==1| dMissed`k'ormore==1
	
	*return the coefficent
	return scalar diff=el(r(table),1,1)
end


cap program drop ttest_beta
program define ttest_beta, rclass
	syntax [, k(integer 3) sim_data(string) command(string) ]
	*Is the hot hand in GVT robust 
quietly{
	clear
	*Load
	use gilovichshooters.dta
	do genregressionvars_v2.do  //generate the variables that are used in the regression
	genregressionvars	
	`command'
	
	inspect sid
	if r(N_unique)> 1{
		xtset sid 
		noi: xtreg make dMade`k'ormore if dMade`k'ormore==1| dMissed`k'ormore==1, fe
	}
	else{
		noi: reg make dMade`k'ormore if dMade`k'ormore==1| dMissed`k'ormore==1
	}
	clear
	use `sim_data'
	summarize, meanonly
	local bias =  r(mean) 
	
	*Alternative Hypothsis 1: Hot or Cold Hand
	noi: test dMade`k'ormore=`bias'
	
	return scalar hot_hand_effect = _b[dMade`k'ormore] - `bias'		
	*Alternative Hypothesis 2: Hot hand (one-sided test, justified by priors?)
		local tstat=sign(_b[dMade`k'ormore])*sqrt(r(F))
		
		return scalar tstat = `tstat'
		return scalar pvalue_right_tail =  ttail(r(df_r) ,`tstat')	
}
end

}

## 2 Simulate coefficient estimates to calculate bias (in the context of fe regression)
This will take 5-10 min

*For sid 109 only, k=3,4
simulate diff=r(diff),reps(10000) saving(MS_Bias_only109_k4.dta): anGVT, fg(design) k(4) command("keep if sid == 109") 
simulate diff=r(diff),reps(10000) saving(MS_Bias_only109_k3.dta): anGVT, fg(design) k(3) command("drop if sid == 109") 

*For GVT without sid 109, k=3,4
simulate diff=r(diff),reps(10000) saving(MS_Bias_without109_k3.dta): anGVT, fg(design) k(3) command("drop if sid == 109")
simulate diff=r(diff),reps(10000) saving(MS_Bias_without109_k4.dta): anGVT, fg(design) k(4) command("drop if sid == 109")

## 3.  How much of a hot hand does shooter 109 have?  A lot

In [8]:
ttest_beta, k(3) sim_data(MS_Bias_only109_k3.dta) command("keep if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



      Source |       SS           df       MS      Number of obs   =        50
-------------+----------------------------------   F(1, 48)        =     15.44
       Model |  2.80333333         1  2.80333333   Prob > F        =    0.0003
    Residual |  8.71666667        48  .181597222   R-squared       =    0.2433
-------------+----------------------------------   Adj R-squared   =    0.2276
       Total |       11.52        49  .235102041   Root MSE        =    .42614

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade3ormore |   .4833333   .1230167     3.93   0.000     .2359917    .7306749
       _cons |        .35   .0952883     3.67   0.001       .15841      .54159
------------------------------------------------------------------------------

 ( 1)  dMade3ormore = -.0844034

       F(  1,  

**Robust to stricter definition of a streak? Length 4 or longer:**

In [9]:
ttest_beta, k(4) sim_data(MS_Bias_only109_k4.dta) command("keep if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



      Source |       SS           df       MS      Number of obs   =        38
-------------+----------------------------------   F(1, 36)        =      7.51
       Model |   1.4757085         1   1.4757085   Prob > F        =    0.0095
    Residual |  7.07692308        36  .196581197   R-squared       =    0.1725
-------------+----------------------------------   Adj R-squared   =    0.1496
       Total |  8.55263158        37  .231152205   Root MSE        =    .44337

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade4ormore |   .4153846   .1516076     2.74   0.010     .1079101    .7228592
       _cons |   .3846154     .12297     3.13   0.003     .1352206    .6340102
------------------------------------------------------------------------------

 ( 1)  dMade4ormore = -.1646685

       F(  1,  

## 4.  Is GVT significant without shooter 109 have?  Yes*, hot hand effect = 10.2pp (SE 4.6)
*Yes for hot hand alternative, borderline for cold or hot hand alternative

In [11]:
ttest_beta, k(3) sim_data(MS_Bias_without109_k3.dta) command("drop if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



Fixed-effects (within) regression               Number of obs     =        663
Group variable: sid                             Number of groups  =         25

R-sq:                                           Obs per group:
     within  = 0.0027                                         min =         13
     between = 0.5772                                         avg =       26.5
     overall = 0.0190                                         max =         38

                                                F(1,637)          =       1.71
corr(u_i, Xb)  = 0.3871                         Prob > F          =     0.1912

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade3ormore |    .059581   .0455326     1.31   0.191    -.0298312    .1489932
       _cons |    .439123   .0273333    16.07   0.000     .3854

**Robust to stricter definition of a streak? Length 4 or longer: hot hand effect = 12.4pp (SE 6.4)

In [12]:
ttest_beta, k(4) sim_data(MS_Bias_without109_k4.dta) command("drop if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



Fixed-effects (within) regression               Number of obs     =        375
Group variable: sid                             Number of groups  =         25

R-sq:                                           Obs per group:
     within  = 0.0012                                         min =          5
     between = 0.4804                                         avg =       15.0
     overall = 0.0268                                         max =         25

                                                F(1,349)          =       0.44
corr(u_i, Xb)  = 0.4644                         Prob > F          =     0.5093

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade4ormore |     .04332   .0655818     0.66   0.509    -.0856653    .1723052
       _cons |   .4171076   .0368046    11.33   0.000      .344